# dev data parsing

In [1]:
from lib.my_initialization import *
# from lib.searchforfile import *

In [2]:
# import pandas as pd, os
def parse_iolines(input_fn, printing=False):
    n_input=-9999
    with open(input_fn) as f:
        trgt1='Printing Inputs...\n'
        trgt2='Printing Outputs...\n'
        for n,line in enumerate(f):
            if trgt1 == line:
                if printing:
                    print(f'found inputs starting after line {n}')
                n_input=n
            if trgt2 == line:
                if printing:
                    print(f'found outputs starting after line {n}')
                n_output=n
    return n_input,n_output

In [3]:
data_dir=f"{nb_dir}/data/osg_output/Log"
os.chdir(data_dir)

input_fn_lst=os.listdir()
input_fn=input_fn_lst[0]
input_fn

'job.out.8872865.6334'

In [4]:
n_input,n_output=parse_iolines(input_fn, printing=False)

In [71]:
def parse_inputs(input_fn,n_input,printing=False):
    if n_input==-9999:
        if printing:
            print(f"Warning: n_input not found for input_fn={input_fn}.\nreturning None...")
        return None
    with open(input_fn) as f:
        inputs=f.readlines()[n_input+1:n_output-1]
    col_name_lst=[]
    col_value_lst=[]
    for line in inputs:
        string=line.split(' ')[-1]
        eid=string.find('=')
        if eid!=-1:
            col_name=string[:eid]
            # try:
            col_value=eval(string[eid+1:-1])
            # except Exception as e:
            #     print (string)
            #     col_value=string[eid+1:-1]
            col_name_lst.append(col_name)
            col_value_lst.append(col_value)
    return col_name_lst, col_value_lst

In [72]:
# nb_dir='/home/timothytyree/Documents/GitHub/bgmc/python/'
data_dir=f"{nb_dir}/data/osg_output/Log"
runtime=compute_runtime_for_folder(folder=data_dir)
print(f"the total recorded runtime was {runtime:.3f} years.")
beep(3)

computing runtime for 40745 files...
the run time for computing runtimes was: 2.81 seconds.
the total recorded runtime was 2.151 years.


0

In [73]:
def parse_output(input_fn, n_output):
    with open ( input_fn ) as f:
        for n,line in enumerate(f):
            if n==n_output+1:
                N_values_string=line
            if n==n_output+2:
                T_values_string=line
    N_values=[eval(s) for s in N_values_string[:-2].split(',')]
    T_values=[eval(s) for s in T_values_string[:-2].split(',')]
    df=pd.DataFrame({'N':N_values,'T':T_values})
    df=df[df['T']>0]
    return df

In [74]:
input_fn_lst=get_files_in_folder(folder=data_dir,trgt='.out.')

In [86]:
def parse_log(input_fn,qfoo,include_inputs=True,printing=False):
    col_name_lst, col_value_lst=parse_inputs(input_fn,n_input,printing=printing)
    dict_inputs=dict(zip(col_name_lst,col_value_lst))
    if not qfoo(dict_inputs):
        return None
    df=parse_output(input_fn, n_output)
    #augment df with input parameters
    if include_inputs:
        if printing:
            print("input parameters were:")
            print(col_name_lst)
            print(col_value_lst)
            print("returning outputs as pandas.DataFrame instance")
        for name,value in zip ( col_name_lst, col_value_lst):
            df[name]=value
    df.drop(columns=['dt','reflect','set_second'],inplace=True)
    return df

In [87]:
input_fn=input_fn_lst[0]
qfoo_pbc=lambda dict_inputs:dict_inputs['reflect']==0 #periodic bc
qfoo_rbc=lambda dict_inputs:dict_inputs['reflect']==1 #reflecting bc
qfoo=qfoo_rbc
df=parse_log(input_fn,qfoo,include_inputs=True,printing=False)

In [88]:
df

,N,T,r,D,L,kappa,beta,niter
11,12,0.000767,0.1,0.2,10,500,1,6334
12,13,0.000646,0.1,0.2,10,500,1,6334
13,14,0.000550,0.1,0.2,10,500,1,6334
14,15,0.000472,0.1,0.2,10,500,1,6334
15,16,0.000410,0.1,0.2,10,500,1,6334
...,...,...,...,...,...,...,...,...
695,696,0.000010,0.1,0.2,10,500,1,6334
696,697,0.000010,0.1,0.2,10,500,1,6334
697,698,0.000010,0.1,0.2,10,500,1,6334
698,699,0.000010,0.1,0.2,10,500,1,6334


In [1]:
# TODO: switch params niter and process in .submit file
# TODO: update github
# TODO: kill grid 
# TODO: restart grid with correct .submit

- TODO: plot the log output over some axis of the input_parameter space  space
- TODO: measure the slope/turning points for a given log output
- TODO: learn the slope/turning points using state space
- TODO: tensorflow jupyter notebook stackoverflow 